In [ ]:
!pip install ultralytics

In [ ]:
!pip install opencv-python

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="coco128.yaml", epochs=3)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
success = model.export(format="onnx")  # export the model to ONNX format

In [2]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
results= model("https://ultralytics.com/images/bus.jpg")


Found https:\ultralytics.com\images\bus.jpg locally at bus.jpg
image 1/1 C:\Users\User\Documents\machine_learning\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 25.0ms
Speed: 6.0ms preprocess, 25.0ms inference, 75.0ms postprocess per image at shape (1, 3, 640, 640)


In [5]:
from PIL import Image
image= Image.open("bus.jpg")
results= model.predict(source=image, save=True)


0: 640x480 4 persons, 1 bus, 1 stop sign, 56.0ms
Speed: 190.3ms preprocess, 56.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict


In [1]:
!yolo mode=predict model=yolov8n.pt source=0

^C


In [1]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO

import supervision as sv


class ObjectDetection:

    def __init__(self, capture_index):
       
        self.capture_index = capture_index
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        
        self.model = self.load_model()
        
        self.CLASS_NAMES_DICT = self.model.model.names
    
        self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=3, text_thickness=3, text_scale=1.5)
    

    def load_model(self):
       
        # model = YOLO("yolov8m.pt")  # load a pretrained YOLOv8n model
        model = YOLO("yolov8n.pt")
        model.fuse()
    
        return model


    def predict(self, frame):
       
        results = self.model(frame)
        
        return results
    

    def plot_bboxes(self, results, frame):
        
        xyxys = []
        confidences = []
        class_ids = []
        
         # Extract detections for person class
        for result in results:
            boxes = result.boxes.cpu().numpy()
            # class_id = boxes.cls[0]
            # conf = boxes.conf[0]
            # xyxy = boxes.xyxy[0]

            if class_ids == 0.0:
          
              xyxys.append(result.boxes.xyxy.cpu().numpy())
              confidences.append(result.boxes.conf.cpu().numpy())
              class_ids.append(result.boxes.cls.cpu().numpy().astype(int))
            
        
        # Setup detections for visualization
        detections = sv.Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                    )
        
    
        # Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
        for _, confidence, class_id, tracker_id
        in detections]
        
        # Annotate and display frame
        frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)
        
        return frame
    
    
    
    def __call__(self):

        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
      
        while True:
          
            start_time = time()
            
            ret, frame = cap.read()
            assert ret
            
            results = self.predict(frame)
            frame = self.plot_bboxes(results, frame)
            
            end_time = time()
            fps = 1/np.round(end_time - start_time, 2)
             
            cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
            
            cv2.imshow('YOLOv8 Detection', frame)
 
            if cv2.waitKey(5) & 0xFF == 27:
                
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
        
    
detector = ObjectDetection(capture_index=0)
detector()

YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


Using Device:  cuda



0: 384x640 (no detections), 14.0ms
Speed: 4.0ms preprocess, 14.0ms inference, 11.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 persons, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 31.0ms postprocess per image at shape (1, 3, 640, 640)


ValueError: too many values to unpack (expected 4)

: 

In [13]:
from ultralytics import YOLO

image = cv2.imread("image1_60_left.jpg")
model = YOLO("best_yolov8.pt")
results = model.predict(image)
result = results[0]


0: 640x512 6 persons, 100.6ms
Speed: 12.4ms preprocess, 100.6ms inference, 14.2ms postprocess per image at shape (1, 3, 640, 512)


In [14]:
len(result.boxes)

6

In [33]:
import random
box = []
for i in range(len(result.boxes)):
    box.append(result.boxes[i])
    class_id, xyxy, xywh = box[i].cls.item() , box[i].xyxy.tolist(), box[i].xywh.tolist()
    x1, y1, x2, y2 = xyxy[0]
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    print(class_id, xyxy, xywh)
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (random.randint(0,255), random.randint(0,255), random.randint(0,255)), 2)
    image = cv2.putText(image, result.names[i], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

0.0 [[547.931640625, 696.5862426757812, 587.9410400390625, 804.6864624023438]] [[567.9363403320312, 750.6363525390625, 40.0093994140625, 108.1002197265625]]
0.0 [[382.3617248535156, 712.8582153320312, 415.1719665527344, 812.1238403320312]] [[398.766845703125, 762.4910278320312, 32.81024169921875, 99.265625]]
0.0 [[434.653076171875, 711.4818115234375, 481.1082458496094, 808.6229248046875]] [[457.88067626953125, 760.0523681640625, 46.455169677734375, 97.14111328125]]
0.0 [[1136.162109375, 727.0494995117188, 1154.9908447265625, 772.88916015625]] [[1145.576416015625, 749.9693603515625, 18.8287353515625, 45.83966064453125]]
0.0 [[483.3175964355469, 725.738525390625, 509.95855712890625, 803.1697387695312]] [[496.6380615234375, 764.4541015625, 26.640960693359375, 77.43121337890625]]
0.0 [[509.769287109375, 729.0890502929688, 539.0842895507812, 803.2787475585938]] [[524.4267578125, 766.1838989257812, 29.31500244140625, 74.189697265625]]


In [29]:
xyxy= xyxy[0]
print(xyxy)

[509.769287109375, 729.0890502929688, 539.0842895507812, 803.2787475585938]


In [19]:
x1,y1,x2,y2 = xyxy

ValueError: not enough values to unpack (expected 4, got 1)

In [15]:
import cv2


NameError: name 'x1' is not defined

In [34]:
cv2.imshow('image',image)

: 